# Version information

In [1]:
from datetime import date
print("Running date:", date.today().strftime("%B %d, %Y"))
import pyleecan
print("Pyleecan version:" + pyleecan.__version__)
import SciDataTool
print("SciDataTool version:" + SciDataTool.__version__)

Running date: March 15, 2022
Pyleecan version:1.3.7
SciDataTool version:1.4.24


# How to solve optimization problem in Pyleecan, using Bayesian optimisation

This tutorial explains how to use Pyleecan to solve **constrained global optimization** problem using bayesian optimization.

The notebook related to this tutorial is available on [GitHub](https://github.com/Eomys/pyleecan/tree/master/Tutorials/).  

The tutorial introduces the different objects to define that enable to parametrize each aspect of the optimization. To do so, we will present an example to maximize the average torque and minimize the ripple torque by varying..

## Problem definition

The object [**OptiProblem**](https://www.pyleecan.org/pyleecan.Classes.OptiObjFunc.html) contains all the optimization problem characteristics:  

- the simulation/machine to iterate on
- the design variable to vary some parameters of the simulation (e.g. input current, topology of the machine)  
- the objective functions to minimize for the simulation  
- some constraints (optional)  

### Reference simulation definition

To define the problem, we first define a reference simulation. Each optimization evaluation will copy the reference simulation, set the value of the design variables and run the new simulation.

This exemple uses the simulation defined in the tutorial [How to define a simulation to call FEMM](https://www.pyleecan.org/tuto_Simulation_FEMM.html), (with less precision for FEMM mesh to speed up the calculations)

In [9]:
from numpy import ones, pi, array, linspace
import numpy as np
from pyleecan.Classes.OPslip import OPslip
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.Output import Output
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.ImportMatrixVal import ImportMatrixVal
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.ImportGenVectLin import ImportGenVectLin
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
from os.path import join

# Import the machine from a script
SCIM_001 = load(join(DATA_DIR, "Machine", "SCIM_001.json"))

# Create the Simulation
simu_ref = Simu1(name="EM_SIPMSM_AL_001", machine=SCIM_001)   

# Defining Simulation Input
Nt = 2
N0 = 3000
Is = ImportMatrixVal(
    value=np.array(
        [
            [6.97244193e-06, 2.25353053e02, -2.25353060e02],
            [-2.60215295e02, 1.30107654e02, 1.30107642e02],
            #             [-6.97244208e-06, -2.25353053e02, 2.25353060e02],
            #             [2.60215295e02, -1.30107654e02, -1.30107642e02],
        ]
    )
)
Ir = ImportMatrixVal(value=np.zeros(30))
time = ImportGenVectLin(start=0, stop=0.015, num=Nt, endpoint=True)
Na_tot = 64

simu_ref.input = InputCurrent(
        Is=Is,
        Ir=Ir,  # zero current for the rotor
        OP=OPslip(N0=N0),
        time=time,
        Na_tot=Na_tot,
        angle_rotor_initial=0.5216 + np.pi,
    )

# Definition of the magnetic simulation (is_mmfr=False => no flux from the magnets)
simu_ref.mag = MagFEMM(
    type_BH_stator=2, # 0 to use the B(H) curve, 
                           # 1 to use linear B(H) curve according to mur_lin,
                           # 2 to enforce infinite permeability (mur_lin =100000)
    type_BH_rotor=2,  # 0 to use the B(H) curve, 
                           # 1 to use linear B(H) curve according to mur_lin,
                           # 2 to enforce infinite permeability (mur_lin =100000)
    file_name = "", # Name of the file to save the FEMM model
    is_periodicity_a=True, # Use Angular periodicity
    is_periodicity_t=True,  # Use time periodicity
)

# We only use the magnetic part 
simu_ref.force = None
simu_ref.struct = None 

### Minimization problem definition

To setup the optimization problem, we define some objective functions using the [**OptiObjective**](https://www.pyleecan.org/pyleecan.Classes.OptiObjective.html) object (which behave the same way as [**DataKeeper**](https://www.pyleecan.org/pyleecan.Classes.DataKeeper.html). 

Each objective function is stored in the *keeper* attribute of a **OptiObjective**. keeper is a function and can be set either with a string (mandatory to be able to save the object) or directly with a function (the function will be discarded when saving). This type of function takes an output object in argument and returns a float to **minimize**. 

We gather the objective functions into a list:

In [10]:
from pyleecan.Classes.OptiObjective import OptiObjective
import numpy as np

objs = [
    OptiObjective(
        symbol="obj1",
        name="Maximization of the torque average",
        keeper="lambda output: output.mag.Tem_av",
    ),
    OptiObjective(
        symbol="obj2",
        name="Minimization of the torque ripple",
        keeper="lambda output: output.mag.Tem_rip_norm",
    ),
]

The first objective is the torque average that is taken from the output.

The second objective is the torque ripple.

### Design variables
We use the object [**OptiDesignVar**](https://www.pyleecan.org/pyleecan.Classes.OptiDesignVar.html) to define the design variables. 


To define a design variable, we have to specify different attributes:  

- *name* to define the design variable name
- *symbol* to access to the variable / for plot (must be unique)
- *unit* to define the variable unit
- *type_var* to specify the variable "type":  
    - *interval* for continuous variables  
    - *set* for discrete variables  
- *space* to set the variable bound
- *setter* to access to the variable in the simu object. This attribute **must begin by "simu"**.  
- *get_value* to define the variable for the first generation, the function takes the space in argument and returns the variable value  

We store the design variables in a dictionnary that will be in argument of the problem. For this example, we define 30 design variables: 

  

In [15]:
from pyleecan.Classes.OptiDesignVar import OptiDesignVar
import random

my_vars = []

def gen_setter(i):
    def new_setter(simu, value):
        simu.input.Ir.value[i] = value

    return new_setter

for i in range(30):
    my_vars.append(
        OptiDesignVar(
            name="Ir({})".format(i),
            symbol="var_" + str(i),
            type_var="interval",
            space=[0, 1],
            get_value=lambda space: np.random.uniform(*space),
            setter=gen_setter(i),
        )
    )




### Constraints

The class [**OptiConstraint**](https://www.pyleecan.org/pyleecan.Classes.OptiConstraint.html) enables to define some constraint. For each constraint, we have to define the following attributes:  

- name  
- type_const: type of constraint  
    - "=="  
    - "<="  
    - "<"  
    - ">="  
    - ">"  
- value: value to compare  
- get_variable: function which takes output in argument and returns the constraint value  

We also store the constraints into a dict.

In [12]:
from pyleecan.Classes.OptiConstraint import OptiConstraint

my_constraint = [
    OptiConstraint(
        name = "const1",
        type_const = "<=",
        value = 700,
        get_variable = "lambda output: abs(output.mag.Tem_rip_pp)",
    )
]

### Evaluation function


We can create our own evaluation function if needed by defining a function which only take an output in argument. 


In [13]:
from pyleecan.Classes.OptiProblem import OptiProblem

# ### Evaluation
def evaluate(output):
    x = output.simu.input.Ir.value
    f1 = lambda x: x[0]
    g = lambda x: 1 + (9 / 29) * np.sum(x[1:])
    h = lambda f1, g: 1 - np.sqrt(f1 / g) - (f1 / g) * np.sin(10 * np.pi * f1)
    output.mag.Tem_av = f1(x)
    output.mag.Tem_rip_norm = g(x) * h(f1(x), g(x))

# ### Defining the problem
my_prob = OptiProblem(
    simu=simu_ref, design_var=my_vars, obj_func=objs, eval_func=evaluate
)

## Solver

Pyleecan separes the problem and solver definition to be able to create different solver that uses the same objects. 

The class [**OptiBayesAlgSmoot**]() enables to solve our problem using Gaussian processes to create a modelization of the machine. This modelization will be cheap to run, and will allow us to perform the [NSGA-II](https://www.iitk.ac.in/kangal/Deb_NSGA-II.pdf) genetical algorithm. The algorithm takes several parameters:  

|Parameter|Description|Type|Default Value|  
| :-: | :- | :-: | :-: |  
|*problem*|Problem to solve|**OptiProblem**|mandatory|
|*nb\_start*|Number of points to start the Gaussian Process|**int**|30|  
|*nb\_iter*|Number of iteration|**int**|5|   
|*nb\_gen*|Generation number|**int**|10|  
|*size\_pop*| Population size per generation|**int**|10| 

  
The `solve` method performs the optimization and returns an [**OutputMultiOpti**](https://www.pyleecan.org/pyleecan.Classes.OutputMultiOpti.html) object which contains the results.

In [14]:
from pyleecan.Classes.OptiBayesAlgSmoot import OptiBayesAlgSmoot
# Solve problem with Bayes
solver = OptiBayesAlgSmoot(problem=my_prob, nb_start=300, nb_iter=10, nb_gen=100, size_pop=40)
res = solver.solve()

19:15:24 Starting optimization...
	Number of iterations: 10
	
iteration 1
PI max value : 0.999999999639857
xopt : [0.62711232 0.53433601 0.39001553 0.295597   0.38692131 0.18111627
 0.31111615 0.57278458 0.09935757 0.21637011 0.59127747 0.57442816
 0.31470187 0.12306473 0.31360171 0.41569652 0.42494431 0.3681749
 0.39682019 0.30615979 0.36691271 0.3600918  0.25179875 0.22305139
 0.38285813 0.18119656 0.44009163 0.2209033  0.36496662 0.44122707]
iteration 2
PI max value : 1.0
xopt : [0.61709891 0.57555442 0.24195363 0.32969536 0.35649541 0.13715362
 0.32538871 0.5151499  0.05944642 0.284804   0.50856317 0.47067819
 0.07500694 0.07181014 0.28010774 0.45441781 0.44998997 0.25047623
 0.39033919 0.28634503 0.40647336 0.40583022 0.31575877 0.2344322
 0.33106972 0.19649769 0.33313271 0.22952262 0.39524182 0.30421332]
iteration 3
PI max value : 0.9999410795437613
xopt : [0.41769095 0.3276337  0.24276271 0.23477825 0.26809574 0.27133048
 0.1535333  0.10060748 0.29949539 0.36237225 0.53790014 0.

During the algorithm the object displays some data containing:

- number of errors: failure during the objective function execution
- number of infeasible: number of individual with constraints violations

## Plot results

**OutputMultiOpti** has several methods to display some results:  

- `plot_generation`: to plot individuals for in 2D  
- `plot_pareto`: to plot the pareto front in 2D    

In [16]:
%matplotlib notebook 
import matplotlib.pyplot as plt 

# Create a figure containing 4 subfigures (axes) 
fig, axs = plt.subplots(2,2, figsize=(8,8))

# Plot every individual in the fitness space 
res.plot_generation(
    x_symbol = "Tem_av", # symbol of the first objective function or design variable
    y_symbol = "Tem_h1", # symbol of the second objective function or design variable
    ax = axs[0,0] # ax to plot
)

# Plot every individual in the design space 
res.plot_generation(
    x_symbol = "SW0", 
    y_symbol = "Rext", 
    ax = axs[0,1]
)

# Plot pareto front in fitness space 
res.plot_pareto(
    x_symbol = "Tem_av", 
    y_symbol = "Tem_h1", 
    ax = axs[1,0]
)

# Plot pareto front in design space 
res.plot_pareto(
    x_symbol = "SW0", 
    y_symbol = "Rext", 
    ax = axs[1,1]
)

fig.tight_layout()


<IPython.core.display.Javascript object>

KeyError: 'ngen'